<a href="https://colab.research.google.com/github/emilymistick/BAIT509/blob/master/Copy_of_Getting_started_with_Brooks_revisited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install

In [0]:
!pip install earthengine-api

Authenticate

In [0]:
# Import the Earth Engine library.
import ee
# Trigger the authentication flow and initialize
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=xTbjxpr_voOoHB1Y_dfHzIow_bOqSs_HVnqfGpW9AG0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/ygExIfMiKnpW3HRDTfT6FNxHE6YODV5eAO6cxzuoAtF9JAMqrpFo7UA

Successfully saved authorization token.


install ipygee using Pip

In [0]:
pip install ipygee

In [0]:
pip install geehydro

In [0]:
!pip install geemap

In [0]:
pip install proplot


     |████████████████████████████████| 7.9MB 2.6MB/s 


Setup

In [0]:
from pprint import pprint  # for pretty-printing data
import altair as alt
import ee
import folium 
import geehydro
import geemap
import numpy as np 
import pandas as pd
import proplot as plot 
import matplotlib.pyplot as plt 
from ipygee import*
from pandas.plotting import register_matplotlib_converters
from folium import plugins

/usr/local/lib/python3.6/dist-packages/proplot/utils.py:105: ProPlotWarning: Rebuilding font cache.


Helper functions

In [0]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)
  
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

Analysis

In [0]:
start_date = ee.Date('2000-03-01')
end_date = ee.Date('2020-02-29')

WS = ee.FeatureCollection('USGS/WBD/2017/HUC08') \
           .filter(ee.Filter.eq('huc8', '02080103')).geometry()
# Define watershed clip function.
def clipWS(image):
   return image.clip(WS)

ndvi = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI') \
         .filterDate(start_date,end_date) \
         .map(clipWS)

Find number of images

In [0]:
print('Number of images:', ndvi.size().getInfo())
print(WS)

Number of images: 460
ee.Geometry({
  "type": "Invocation",
  "arguments": {
    "collection": {
      "type": "Invocation",
      "arguments": {
        "collection": {
          "type": "Invocation",
          "arguments": {
            "tableId": "USGS/WBD/2017/HUC08"
          },
          "functionName": "Collection.loadTable"
        },
        "filter": {
          "type": "Invocation",
          "arguments": {
            "leftField": "huc8",
            "rightValue": "02080103"
          },
          "functionName": "Filter.equals"
        }
      },
      "functionName": "Collection.filter"
    }
  },
  "functionName": "Collection.geometry"
})


Visualize

In [0]:
# Define RGB visualization parameters. From https://developers.google.com/earth-engine/tutorials/community/modis-ndvi-time-series-animation
visParams = {
  'min': 0.0,
  'max': 9000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', \
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01', \
    '012E01', '011D01', '011301'],}

# Create a folium map object.
Map = folium.Map(location=[38.5, -78], zoom_start=5)
Map.setOptions('HYBRID')
Map.centerObject(WS,7)
Map.addLayer(ndvi.first(),visParams,'NDVI.first')
Map

# Plotting summary data

Make a chart of mean NDVI values for the watershed

In [0]:
# NDVI time series
WS_ndvi = chart.Image.series(**{'imageCollection': ndvi,
                                   'region': WS,
                                   'reducer': ee.Reducer.mean(),
                                   'scale': 1000,
                                   'xProperty': 'system:time_start'})
WS_ndvi.renderWidget(width='50%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…